In [6]:
# filter tickets by:
# project = "AWS Loc-2023"  AND issuetype in ("AWS ADHOC", "AWS Blog", "AWS Daily Posts", "AWS KB", "AWS SW", "AWS WP") AND summary ~ "AWS-HO*" AND created > "2023/1/2 00:54" AND created < "2023/1/4 22:00" ORDER BY summary ASC      

In [12]:
import pandas as pd
import xlsxwriter
import numpy as np
import os
import time
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
#startTime
startTime = time.time()

pd.set_option("display.max_columns", 100) #local settings 

driver=webdriver.Chrome()
driver.get("file:///C:/Users/home/TEMP/LOC/jira_cut.html");
driver.maximize_window();

summary_data = driver.find_elements("xpath", '//table[@class="tableBorder"]/tbody/tr/td/h3/a') 
lang_code = driver.find_elements("xpath", '//div[@class="table-wrap"]/table/tbody/tr[count(*)>3]/td[1]')
supp_id = driver.find_elements("xpath", '//div[@class="table-wrap"]/table/tbody/tr[count(*)>3]/td[2]/b[font] | //div[@class="table-wrap"]/table/tbody/tr[count(*)>3]/td[2]/font[b] | //div[@class="table-wrap"]/table/tbody/tr[count(*)>3]/td[position()=2 and not(b[font] or font[b])]')
dates = driver.find_elements("xpath", '//div[@class="table-wrap"]/table/tbody/tr[count(*)>3]/td[3]')
comments = driver.find_elements("xpath", '//div[@class="table-wrap"]/table/tbody/tr[count(*)>3]/td[4]')

# table_data; DF with parsed data
fetched_data = [{'Language_Code': lc.text,
                 'Supplier_ID': si.text,
                 'Dates': d.text,
                 'Comments': c.text} for lc, si, d, c in zip(lang_code, supp_id, dates, comments)]
table_data = pd.DataFrame(fetched_data)
table_data = table_data.applymap(lambda x: x.strip() if isinstance(x, str) else x) # strip

# df with parsed "Summary" data (e.g AWS-HO# | PID# | Projname | Term)
summary_data_array = [{'Summary': s.text} for s in summary_data]
summary_data_df = pd.DataFrame(summary_data_array)
summary_data_df = summary_data_df.applymap(lambda x: x.strip() if isinstance(x, str) else x) # strip

# converts to datetime + skips incorrect date insertion in JIRA
table_data['Dates'] = pd.to_datetime(table_data['Dates'], errors='coerce').dt.strftime('%m/%d/%Y')

# list for replacements 
repl = ['Lang Code', 'Supplier ID', 'TEP Deadline', 'Comment, Description, Status', 'Code', 'Lang. code', 'STATUS', 'Vendor ID', 'STATUS']
# use "repl" list of strings, find them in columns below, replace them with np.nan;   
table_data[['Language_Code', 'Supplier_ID', 'Dates', 'Comments']] = table_data[['Language_Code', 'Supplier_ID', 'Dates', 'Comments']].replace(repl, np.nan)

# below will create "group" column; new number will trigger every time it encounters row with np.nans
# with this, "group" col will separate each ticket by unique number
table_data['group'] = table_data['Language_Code'].isna().cumsum()
table_data = table_data.sort_values(['group', 'Language_Code'], ascending=True) 

# split summary (e.g AWS-HO# | PID# | Projname | Term) 
delimiter_split_df = summary_data_df['Summary'].str.split(" | ", expand=True)
# DF with AWS-HOs for mapping
map_df = delimiter_split_df[[0]].rename(columns={0: 'AWS-HO'})
# # DF with Proj_name for mapping
proj_name = delimiter_split_df[[4]].rename(columns={4: 'Project_name'}).assign(name=lambda x: x['Project_name'].str.strip().str.replace('| ', ''))

# map with AWS-HOs & Proj_names
table_data["AWS-HO"] = table_data.groupby("group").ngroup().map(map_df["AWS-HO"]) #map
table_data["Project_name"] = table_data.groupby("group").ngroup().map(proj_name["Project_name"])

# converts parsed Supplier IDs to Supplier names; takes data from "internal_LOC.xlsx"
internal = pd.read_excel('./internal_files/internal_LOC.xlsx', sheet_name='SuppIDs') #sheet with suppIDs and names
internal_dict = dict(zip(internal['Supplier ID'], internal['Supplier name'])) #creates a dictionary for map
table_data["Supplier_Name"] = table_data['Supplier_ID'].map(internal_dict) #map

#
table_data = table_data[['group', 'AWS-HO', 'Project_name', 'Language_Code', 'Supplier_ID', 'Supplier_Name', 'Dates', 'Comments']]
table_data = table_data.dropna(thresh=4) #gets rid of empty lines

# repl. #2; drops rows, that need to be ignored
repl2 = ['ar-SA (Review)', 'ar-MSA (Review )', 'ar-MSA (Review)', 'ar-MSA ( Review )', 'ar-MSA(ER)', 'ar-MSA (ER)', 'ar-MSA ( Review)',
         'ar-SA-Review', 'ar-SA-Review', 'ar-MSA (REVIEW)', 'ar-MSA ( Review)', 'ar-MSA (REVIEW)', 'ar-SA (REVIEW)',
        'es-ES (MTPE-L)', 'it-IT (MTPE-L)', 'en-GB >it-IT', 'ar-ASA Review', 'ar-MSA ( Review)', 'ar-MSA (Extra Review)',
        'ar-MSA (Extra Review)', 'ar-MSA (Ext Review)', 'ar-MSA ER'] 

table_data[['Language_Code']] = table_data[['Language_Code']].replace(repl2, 'IGNORE') #fix ar-SA Review lines with replace func (various variabilities)
table_data = table_data[table_data['Language_Code'] != "IGNORE"]

# saves 
parsed_cols = table_data[['AWS-HO', 'Project_name', 'Supplier_ID', 'Supplier_Name', 'Dates', 'Comments']]

parsed_cols['Dates'] = parsed_cols['Dates'].astype('datetime64[ns]')

# if "fully leveraged" in Comment section = adds INTERNAL (no loc) to Supplier_Name col. If not = leaves blank
prefixes = ['fully', 'FULLY', 'Fully', 'Fixes done internally', 'fixes done internally', 'Internally fixed', 'INTERN']

parsed_cols['Supplier_Name'] = np.where(parsed_cols['Comments'].str.startswith(tuple(prefixes)), 'INTERNAL (no loc)',
                                        parsed_cols['Supplier_Name'])

parsed_cols['Supplier_ID'] = np.where(parsed_cols['Supplier_Name'] == "INTERNAL (no loc)", 'Internal', 
                                      parsed_cols['Supplier_ID'])

parsed_cols.to_excel('Supplier_IDs.xlsx', sheet_name='PROJECT DETAILS', index=False)


#driver.quit()




num_of_entries = parsed_cols['Supplier_ID'].count()
print(str(num_of_entries) + " rows have been parsed")

executionTime = round((time.time() - startTime), 2)
print('Execution time: ' + str(executionTime) + ' sec.')

42 rows have been parsed
Execution time: 6.89 sec.


In [9]:
delimiter_split_df

0  1                   2  3  \
0  AWS-HO2070  |  PID1123537-1123555  |   
1  AWS-HO2071  |  PID1123556-1123571  |   
2  AWS-HO2072  |  PID1123563-1123579  |   
3  AWS-HO2073  |  PID1123578-1123589  |   

                                              4  
0          AWS-SW-CON-AppMesh-59f77b5f.23.02.27  
1        AWS-SW-CON-EMRStudio-0208e1d0.23.02.27  
2              AWS-SW-CON-VPC-42e13bf9.23.02.27  
3  AWS-SW-CON-AWSLockeConsole-b4b850d3.23.02.27

In [11]:
table_data

group      AWS-HO                                  Project_name  \
1       1  AWS-HO2070          AWS-SW-CON-AppMesh-59f77b5f.23.02.27   
11      1  AWS-HO2070          AWS-SW-CON-AppMesh-59f77b5f.23.02.27   
2       1  AWS-HO2070          AWS-SW-CON-AppMesh-59f77b5f.23.02.27   
3       1  AWS-HO2070          AWS-SW-CON-AppMesh-59f77b5f.23.02.27   
4       1  AWS-HO2070          AWS-SW-CON-AppMesh-59f77b5f.23.02.27   
5       1  AWS-HO2070          AWS-SW-CON-AppMesh-59f77b5f.23.02.27   
6       1  AWS-HO2070          AWS-SW-CON-AppMesh-59f77b5f.23.02.27   
7       1  AWS-HO2070          AWS-SW-CON-AppMesh-59f77b5f.23.02.27   
8       1  AWS-HO2070          AWS-SW-CON-AppMesh-59f77b5f.23.02.27   
9       1  AWS-HO2070          AWS-SW-CON-AppMesh-59f77b5f.23.02.27   
10      1  AWS-HO2070          AWS-SW-CON-AppMesh-59f77b5f.23.02.27   
13      2  AWS-HO2071        AWS-SW-CON-EMRStudio-0208e1d0.23.02.27   
23      2  AWS-HO2071        AWS-SW-CON-EMRStudio-0208e1d0.23.02.27   
14      2  AWS-HO2071        AWS-SW-CON-EMRStudio-0208e1d0.23.02.27   
15      2  AWS-HO2071        AWS-SW-CON-EMRStudio-0208e1d0.23.02.27   
16      2  AWS-HO2071        AWS-SW-CON-EMRStudio-0208e1d0.23.02.27   
17      2  AWS-HO2071        AWS-SW-CON-EMRStudio-0208e1d0.23.02.27   
18      2  AWS-HO2071        AWS-SW-CON-EMRStudio-0208e1d0.23.02.27   
19      2  AWS-HO2071        AWS-SW-CON-EMRStudio-0208e1d0.23.02.27   
20      2  AWS-HO2071        AWS-SW-CON-EMRStudio-0208e1d0.23.02.27   
21      2  AWS-HO2071        AWS-SW-CON-EMRStudio-0208e1d0.23.02.27   
22      2  AWS-HO2071        AWS-SW-CON-EMRStudio-0208e1d0.23.02.27   
25      3  AWS-HO2072              AWS-SW-CON-VPC-42e13bf9.23.02.27   
26      3  AWS-HO2072              AWS-SW-CON-VPC-42e13bf9.23.02.27   
27      3  AWS-HO2072              AWS-SW-CON-VPC-42e13bf9.23.02.27   
28      3  AWS-HO2072              AWS-SW-CON-VPC-42e13bf9.23.02.27   
29      3  AWS-HO2072              AWS-SW-CON-VPC-42e13bf9.23.02.27   
30      3  AWS-HO2072              AWS-SW-CON-VPC-42e13bf9.23.02.27   
31      3  AWS-HO2072              AWS-SW-CON-VPC-42e13bf9.23.02.27   
32      3  AWS-HO2072              AWS-SW-CON-VPC-42e13bf9.23.02.27   
33      3  AWS-HO2072              AWS-SW-CON-VPC-42e13bf9.23.02.27   
34      3  AWS-HO2072              AWS-SW-CON-VPC-42e13bf9.23.02.27   
36      4  AWS-HO2073  AWS-SW-CON-AWSLockeConsole-b4b850d3.23.02.27   
45      4  AWS-HO2073  AWS-SW-CON-AWSLockeConsole-b4b850d3.23.02.27   
37      4  AWS-HO2073  AWS-SW-CON-AWSLockeConsole-b4b850d3.23.02.27   
38      4  AWS-HO2073  AWS-SW-CON-AWSLockeConsole-b4b850d3.23.02.27   
39      4  AWS-HO2073  AWS-SW-CON-AWSLockeConsole-b4b850d3.23.02.27   
40      4  AWS-HO2073  AWS-SW-CON-AWSLockeConsole-b4b850d3.23.02.27   
41      4  AWS-HO2073  AWS-SW-CON-AWSLockeConsole-b4b850d3.23.02.27   
42      4  AWS-HO2073  AWS-SW-CON-AWSLockeConsole-b4b850d3.23.02.27   
43      4  AWS-HO2073  AWS-SW-CON-AWSLockeConsole-b4b850d3.23.02.27   
44      4  AWS-HO2073  AWS-SW-CON-AWSLockeConsole-b4b850d3.23.02.27   

   Language_Code Supplier_ID                         Supplier_Name  \
1          de-DE    COLL0024                       COLLEGANDO SRLS   
11         en-GB    EMMA4501                        Emma Greenwood   
2          es-XN    ARCA0002                  Arcadia Translations   
3          fr-FR    ALAN9999                       INTERNAL (Alan)   
4          id-ID    DRLO0001  Dr.Localize Communication Bridge Co.   
5          it-IT    BABI0002                               Babilon   
6          ja-JP    ICOS0001                              ICOS INC   
7          ko-KR    ENKO0001                              ENKOLINE   
8          pt-BR    FOLL0001                             Follow-Up   
9          zh-CN    ASIA0013      Asian Union Technologies Co.,Ltd   
10         zh-TW    LOCA0014                  M3 Localization Ltd.   
13         de-DE    COLL0024                       COLLEGANDO SRLS   
23         en-GB    EMMA4501            

In [ ]:
parsed_cols

In [ ]:
#JIRA EXPORT PROJECT NAMES LIST

import pandas as pd
df = pd.read_excel('JIRA.xls')
df.columns = df.loc[1]
df = df[2:]
df.reset_index(drop=True, inplace=True)

df['Summary'] = df['Summary'].str.replace(' ', '')
df = df['Summary'].str.split('|', expand=True)

df[2] = np.where(df[2] == "|", df[3],
                np.where(df[2] == '', df[3],
                         np.where(df[2] == '', df[1], df[2])))

df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

df.to_excel('Project_names_extracted2.xlsx', index=False)